In [1]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

class ResNet1d(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet1d, self).__init__()
        self.in_channels = 16
        self.conv = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.bn = nn.BatchNorm1d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                nn.Conv1d(self.in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
                nn.BatchNorm1d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [2]:
import torch
import torch.nn as nn
import numpy as np

# Create an instance of the ResNet1d model
model = ResNet1d(ResidualBlock, [2, 2, 2], num_classes=10)

# Generate a random input tensor of shape (batch_size, 1, input_length)
input_tensor = torch.from_numpy(np.random.randn(64, 1, 100)).float()

# Pass the input tensor through the model
output = model(input_tensor)

# Print the shape of the output tensor
print(output.shape)

torch.Size([64, 10])


In [3]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(input_size, hidden_size, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(hidden_size)

    def forward(self, x):
        residual = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = x + residual
        x = self.relu(x)
        return x

class ResNet1D(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_residual_blocks):
        super(ResNet1D, self).__init__()
        self.conv = nn.Conv1d(input_size, hidden_size, kernel_size=7, stride=2, padding=3)
        self.bn = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.residual_blocks = nn.ModuleList([ResidualBlock(hidden_size, hidden_size) for _ in range(num_residual_blocks)])
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        for residual_block in self.residual_blocks:
            x = residual_block(x)
        x = x.mean(dim=-1)
        x = self.fc(x)
        return x

model = ResNet1D(input_size=3, hidden_size=32, output_size=10, num_residual_blocks=3)

In [5]:
model(torch.randn(1,3,32))

tensor([[-0.1381, -1.1089,  0.7113, -0.5685, -0.0908, -1.3627,  1.0142,  0.5564,
          0.0166,  0.3591]], grad_fn=<AddmmBackward0>)

In [7]:
model.conv

Conv1d(3, 32, kernel_size=(7,), stride=(2,), padding=(3,))